# 量化

直觉（以float32到int8的量化为例）: 
1. 把[float32](https://www.h-schmidt.net/FloatConverter/IEEE754.html)的范围直接变换到int8
   |数据类型|数据点|数据范围|
   |-|-|-|
   |float32|2^32|[−3.4028235×1038,−1.4013×10−45]∪[1.4013×10−45,3.4028235×1038]|
   |int8|2^8|[−128,127]|

<center class='img'>
<img src="assets/img/most_general_quant.png" style="zoom: 40%;">
</center>